In [1]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

In [2]:
import pandas as pd

data = pd.read_csv('checkins.csv')

In [3]:
data.shape

(396634, 6)

In [5]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [10]:
new_data = data[:100000].drop(['venue_id', 'user_id', 'created_at', 'id'], axis=1)

In [11]:
new_data.shape

(100000, 2)

In [27]:
%%time
from sklearn.cluster import MeanShift

clustering = MeanShift(bandwidth=0.1).fit(new_data)

CPU times: user 6min 21s, sys: 13.2 s, total: 6min 34s
Wall time: 12min 44s


In [28]:
clustering.labels_

array([ 5,  7, 30, ..., 25, 19,  4])

In [29]:
cnt = {}

for x in range(new_data.shape[0]):
    row = clustering.labels_[x]
    if row in cnt:
        cnt[row] += 1
    else:
        cnt[row] = 0

In [57]:
clusters = []

for x,y in cnt.items():
    if y > 15:
        clusters.append(x)

In [58]:
cluster_centers = list(map(lambda x: (x[0], x[1]), clustering.cluster_centers_))

In [59]:
offices = [(33.751277, -118.188740), (25.867736, -80.324116),
            (51.503016, -0.075479), (52.378894, 4.885084),
            (39.366487, 117.036146), (-33.868457, 151.205134)]

In [63]:
import math

def distance(x, y):
    return math.sqrt((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2)

In [64]:
answers = []

for x in clusters:
    min_d = 10e9
    for y in offices:
        dist = distance(cluster_centers[x], y)
        if dist < min_d:
            min_d = dist
    answers.append((min_d, x))
    
answers.sort()

ans = answers[0][1]

print(ans)

420


In [65]:
coords = cluster_centers[ans]

with open('ans.txt', 'w') as fl:
    fl.write(f'{coords[0]} {coords[1]}')

In [66]:
print(answers[0])

(0.007834758163107856, 420)
